<a href="https://colab.research.google.com/github/hwangsae91/project_so/blob/master/exploration/221011/%5BExp_08%5D20221011.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# google colab전용
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Mecab 설치를 위한 환경구성

import os
# install konlpy, jdk, JPype
!pip install konlpy
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip3 install JPype1-py3

# install mecab-ko
os.chdir('/tmp/')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
!tar zxfv mecab-0.996-ko-0.9.2.tar.gz
os.chdir('/tmp/mecab-0.996-ko-0.9.2')
!./configure
!make
!make check
!make install

# install mecab-ko-dic
!apt-get install automake
os.chdir('/tmp')
!curl -LO https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
!tar -zxvf mecab-ko-dic-2.1.1-20180720.tar.gz
os.chdir('/tmp/mecab-ko-dic-2.1.1-20180720')
!./autogen.sh
!./configure
!make
!make install

# install mecab-python
os.chdir('/content')
!git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
os.chdir('/content/mecab-python-0.996')
!python3 setup.py build
!python3 setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  2116k      0 --:--:-- --:--:-- --:--:-- 2116k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.9

# exploration 8번째 과제</br>
@ 황한용(3기/쏘카)

## 라이브러리 선언

In [3]:
import pandas as pd
import konlpy
import gensim
import numpy as np

from collections import Counter
from konlpy.tag import Mecab
from pathlib import Path
from typing import Dict, Tuple

## 상수선언

In [7]:
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/data/sentiment_classification" # 데이터 기본경로
DATA_PATH  = BASE_PATH + "/data"
MODEL_PATH = BASE_PATH + "/model"
STEP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어
MAX_NUM_WORDS = 10000 # 최대 사전 단어갯수
FEATURE_DATA = ["document"]
tokenizer = Mecab()

## 함수

In [5]:
def load_data(
    train_data:pd.DataFrame
    , test_data:pd.DataFrame
    , step_words:list=STEP_WORDS
    , num_words:int=MAX_NUM_WORDS
) -> Tuple[
            np.ndarray
            , np.ndarray
            , np.ndarray
            , np.ndarray
            , Dict[int,str]
        ]:
    """
    다음과 같은 전처리 후
    `train data(feature, target)`, `test data(feature, target)`, `단어사전`
    을 리턴한다.

    - 데이터의 중복 제거
    - `NaN` 결측치 제거
    - 한국어 토크나이저로 토큰화
    - 불용어(`stop_words`) 제거
    - 사전`word_to_index` 구성
    - 텍스트 스트링을 사전 인덱스 스트링으로 변환

    Parameters
    ----------
    train_data : DataFrame
        학습 데이터
    train_data : DataFrame
        테스트 데이터
    step_words : list, default = `MAX_NUM_WORDS`
        불용어
    num_words : int, default = `MAX_NUM_WORDS`
        단어사전의 최대 단어 갯수

    Returns
    -------
    X_train : ndarray
        train feature data
    y_train : ndarray
        train target data
    X_test : ndarray
        test feature data
    y_test : ndarray
        test target data
    word_to_index : dict
        단어사전
    """
    # 1.
    train_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)
    test_data.drop_duplicates(subset=FEATURE_DATA, inplace=True)

    # 2.
    train_data.dropna(how='any', inplace=True)
    test_data.dropna(how='any', inplace=True)

    # 3.
    X_train = [
        [word for word in tokenizer.morphs(sentence) if not word in step_words]
        for sentence in train_data['document']
    ]

    X_test = [
        [word for word in tokenizer.morphs(sentence) if not word in step_words]
        for sentence in test_data['document']
    ]
    
    # 4.
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(num_words-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]

    # 5.
    word_to_index = {word:index for index, word in enumerate(vocab)}
        
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
        
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
        
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

## 메인

In [8]:
train_data = pd.read_table(DATA_PATH + "/ratings_train.txt")
test_data = pd.read_table(DATA_PATH + "/ratings_test.txt")
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터에 대한 설명은 아래와 같다.</br>
- id: The review id, provieded by Naver
- document: The actual review
- label: The sentiment class of the review. (0: negative, 1: positive)

In [9]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

9997
